# Домашнее задание № 3

**Предметная область:** медицинская клиника.

**Дано:**
* файл с результатами анализов в зашифрованном виде - medicine.xlsx (файл располагается в одном каталоге с итоговой программой).\
* таблицы med_an_name, med_name для расшифровки показаний (в рамках задания создайте их в своей базе данных, код ниже). 

**Что необходимо сделать:**
1) Режим попроще:
    * забираете данные с листа `easy`
    * вычисляете пациентов, у которых не в порядке хотябы один анализ
    * сохраняете в xlsx имя, телефон, название анализа и заключение: 'Повышен' или 'Понижен'

2) Режим посложнее:
    * забираете данные с листа `hard`
    * вычисляете пациентов, у которых не в порядке два и более анализов
    * сохраняете в xlsx имя, телефон, название анализа и заключение: 'Повышен', 'Понижен' или 'Положительный'

**Что ожидается на выходе:**\
В ответе приложите файл - скрипт python (название easy.py или hard.py в зависимости от выбранного режима)\
Результатом работы скрипта должен быть xlsx файл с заданной информацией.

#### *Создание таблицы с медицинскими показателями*

In [ ]:
CREATE TABLE med_an_name (
	id varchar(10) NULL,
	name varchar(200) NULL,
	is_simple bpchar(1) NULL,
	min_value numeric(18, 2) NULL,
	max_value numeric(18, 2) NULL
);

INSERT INTO med_an_name (id,name,is_simple,min_value,max_value) VALUES
	 ('IG','Иммуноглобулин общий','N',0.00,100.00),
	 ('ALAT','Аланинаминотрансфераза ','N',0.00,45.00),
	 ('2-A','Лейкоциты (моча)','N',0.00,5.00),
	 ('G124','Гаптоглобин','N',150.00,2000.00),
	 ('890','Глюкоза','N',3.30,5.50),
	 ('S','Щелочная фосфатаза','N',40.00,150.00),
	 ('1-100','Общий белок ','N',64.00,84.00),
	 ('N','Нитрит (моча)','Y',NULL,NULL),
	 ('3-511','Креатинин','N',62.00,115.00),
	 ('Z','Эритроциты (моча)','N',0.00,2.00),
	 ('IRR','Липаза','N',0.00,190.00),
	 ('GLK','Глюкоза (моча)','Y',NULL,NULL),
	 ('1-875','Грибки (моча)','Y',NULL,NULL),
	 ('1-900','Амилаза панкреатическая','N',0.00,50.00),
	 ('BBB','Белок (моча)','Y',NULL,NULL),
	 ('AU','Гематокрит','N',39.00,49.00),
	 ('C','СОЭ','N',2.00,20.00),
	 ('aZz','Билирубин','N',5.00,20.00);

#### *Создание таблицы с информацией о клиентах*

In [ ]:
CREATE TABLE med_name (
	id int4 NULL,
	name varchar(50) NULL,
	phone varchar(20) NULL
);

INSERT INTO med_name (id,name,phone) VALUES
	 (191,'Чемиренко Д.А','+7 (905) 912-28-47'),
	 (140,'Головцов Р.Р','+7 (906) 949-98-17'),
	 (119,'Сакобов А.Т','+7 (952) 996-60-21'),
	 (57,'Тарлов И.Е','+7 (900) 875-67-38'),
	 (195,'Свенюков Б.Ю','+7 (983) 577-27-21'),
	 (53,'Попелицкий Р.Г','+7 (976) 841-79-95'),
	 (96,'Умралиев Р.Э','+7 (941) 954-55-88'),
	 (62,'Бондарев С.Г','+7 (906) 933-95-63'),
	 (199,'Цетнарский Э.И','+7 (908) 245-80-47'),
	 (114,'Даклеев Т.М','+7 (925) 612-93-95'),
	 (73,'Гармокацкий С.Л','+7 (935) 653-32-78'),
	 (80,'Маржецкий А.Я','+7 (949) 912-97-84'),
	 (151,'Поляховский Г.Д','+7 (998) 596-45-47'),
	 (127,'Галендук Л.Г','+7 (935) 164-87-46'),
	 (135,'Номоконов Г.Р','+7 (940) 384-74-90'),
	 (77,'Шелкоплясов Э.В','+7 (979) 643-71-75'),
	 (67,'Эльменькин Д.Р','+7 (982) 643-36-73'),
	 (176,'Ламок А.О','+7 (984) 779-54-57'),
	 (163,'Бочечкаров Э.Д','+7 (943) 102-79-69'),
	 (192,'Голяминских Г.Р','+7 (903) 280-60-58'),
	 (72,'Пахарев Ю.Э','+7 (977) 810-48-75'),
	 (111,'Ведмидев Я.У','+7 (997) 541-22-17'),
	 (118,'Сисемкин В.Р','+7 (960) 542-96-83'),
	 (148,'Штыкулин И.А','+7 (949) 862-47-83'),
	 (162,'Шапаев Л.А','+7 (919) 109-43-56'),
	 (76,'Млицкий С.А','+7 (987) 542-73-38'),
	 (101,'Загатин А.Д','+7 (987) 928-64-89');

#### *Решение*

In [ ]:
## Решение Easy
## Сразу после него внизу будет Решение Hard

import pandas as pd
import psycopg2
from psycopg2 import Error

# Параметры подключения к БД (ЗАМЕНИТЕ НА ВАШИ)
DB_NAME = 'postgres'
DB_USER = 'postgres'
DB_PASSWORD = 'postgres'
DB_HOST ='localhost'
DB_PORT = '5433'

INPUT_EXCEL_FILE = 'medicine.xlsx'
INPUT_SHEET_NAME = 'easy'
OUTPUT_EXCEL_FILE = 'result_easy.xlsx'

def get_db_connection():
    # Устанавливаем соединение с базой данных PostgreSQL
    conn = None
    try:
        conn = psycopg2.connect(
            database = DB_NAME,
            user = DB_USER,
            password = DB_PASSWORD,
            host = DB_HOST,
            port = DB_PORT,
        )
        print('Соединение с базой данных PostgreSQL установлено')
    except Error as e:
        print(f'Ошибка при подключении к PostgreSQL: {e}')
    return conn

def main():
    conn = get_db_connection()
    if not conn:
        return
    try:
        # Чтение данных из Excel файла
        df_medicine = pd.read_excel(INPUT_EXCEL_FILE, sheet_name=INPUT_SHEET_NAME)

        # Чтение данных из таблиц БД
        df_med_an_name = pd.read_sql('SELECT id, name, is_simple, min_value, max_value FROM med_an_name', conn)
        df_med_name = pd.read_sql('SELECT id, name, phone FROM med_name', conn)
        # Заменим все NaN на 0
        df_med_an_name = df_med_an_name.fillna(0)
        
        # Переименуем названия столбцов в таблицах
        df_med_an_name = df_med_an_name.rename(columns={'id': 'Анализ'})
        df_med_an_name = df_med_an_name.rename(columns={'name': 'Название анализа'})
        df_med_name = df_med_name.rename(columns={'id': 'Код пациента'})
        
        # Объединим все три таблицы в одну
        df_merged = pd.merge(df_medicine, df_med_an_name, on='Анализ', how='left')
        df_merged = pd.merge(df_merged, df_med_name, on='Код пациента', how='left')
        
        # Создадим таблицы с пациентами, у которых повышенные и пониженные анализы
        df_enhanced_analysis = df_merged[df_merged['Значение'] > df_merged['max_value']]
        df_reduced_analysis = df_merged[df_merged['Значение'] < df_merged['min_value']]

        # Создадим список, в который занесем результаты анализов из предыдущих двух таблиц 
        result_analysis = []
        for _, row in df_enhanced_analysis.iterrows():
            result_analysis.append('Повышен')
        for _, row in df_reduced_analysis.iterrows():
            result_analysis.append('Понижен')

        # Объединим таблицы с повышенными и пониженными результатами анализов в одну таблицу
        # У нас получилась таблица с пациентами, у которых не в порядке анализы
        df_total_analysis = pd.concat([df_enhanced_analysis, df_reduced_analysis])
        # Добавим к ней столбец с результатами анализов
        df_total_analysis['Заключение'] = result_analysis 

        # Собираем итоговый DataFrame
        output_data = []
        for _, row in df_total_analysis.iterrows():
            output_data.append({
                'ФИО пациента': row['name'],
                'Телефон': row['phone'],
                'Название анализа': row['Название анализа'],
                'Заключение': row['Заключение']
            })

        df_output = pd.DataFrame(output_data)

        # Отсортируем фамилии пациентов по алфавиту
        df_output = df_output.sort_values('ФИО пациента')

        # Сохранение результата в Excel
        df_output.to_excel(OUTPUT_EXCEL_FILE, index=False)
        print(f"Результат сохранен в файл: {OUTPUT_EXCEL_FILE}")

    except FileNotFoundError:
        print(f"Ошибка: Файл {INPUT_EXCEL_FILE} не найден.")
    except Error as e:
        print(f"Ошибка при работе с базой данных: {e}")
    except Exception as e:
        print(f"Произошла непредвиденная ошибка: {e}")
    finally:
        # Закрытие соединения
        if conn:
            conn.close()
            print("Соединение с PostgreSQL закрыто")

if  __name__ == "__main__":
    main()



Соединение с базой данных PostgreSQL установлено
Результат сохранен в файл: result_easy.xlsx
Соединение с PostgreSQL закрыто


C:\Users\Maks\AppData\Local\Temp\ipykernel_9616\3319920853.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_med_an_name = pd.read_sql('SELECT id, name, is_simple, min_value, max_value FROM med_an_name', conn)
C:\Users\Maks\AppData\Local\Temp\ipykernel_9616\3319920853.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_med_name = pd.read_sql('SELECT id, name, phone FROM med_name', conn)


In [27]:
## Решение Hard

import pandas as pd
import psycopg2
from psycopg2 import Error

# Параметры подключения к БД (ЗАМЕНИТЕ НА ВАШИ)
DB_NAME = 'postgres'
DB_USER = 'postgres'
DB_PASSWORD = 'postgres'
DB_HOST ='localhost'
DB_PORT = '5433'

INPUT_EXCEL_FILE = 'medicine.xlsx'
INPUT_SHEET_NAME = 'hard'
OUTPUT_EXCEL_FILE = 'result_hard.xlsx'

def get_db_connection():
    # Устанавливаем соединение с базой данных PostgreSQL
    conn = None
    try:
        conn = psycopg2.connect(
            database = DB_NAME,
            user = DB_USER,
            password = DB_PASSWORD,
            host = DB_HOST,
            port = DB_PORT
        )
        print('Соединение с базой данных PostgreSQL установлено')
    except Error as e:
        print(f'Ошибка при подключении к PostgreSQL: {e}')
    return conn

def main():
    conn = get_db_connection()
    if not conn:
        return
    
    try:
        # Чтение данных из Excel файла
        df_medicine = pd.read_excel(INPUT_EXCEL_FILE, sheet_name=INPUT_SHEET_NAME)

        # Чтение данных из таблиц БД и замена всех NaN на 0
        df_med_an_name = pd.read_sql('SELECT id, name, is_simple, min_value, max_value FROM med_an_name', conn).fillna(0)
        df_med_name = pd.read_sql('SELECT id, name, phone FROM med_name', conn)
        
        # Переименуем названия столбцов в таблицах
        df_med_an_name = df_med_an_name.rename(columns={'id': 'Анализ'})
        df_med_an_name = df_med_an_name.rename(columns={'name': 'Название анализа'})
        df_med_name = df_med_name.rename(columns={'id': 'Код пациента'})
        
        # Объединим все три таблицы в одну
        df_merged = pd.merge(df_medicine, df_med_an_name, on='Анализ', how='inner')
        df_merged = pd.merge(df_merged, df_med_name, on='Код пациента', how='inner')
        
        # Создадим таблицу с пациентами, у которых положительные анализы
        df_positive_analysis = df_merged[df_merged['Значение'].isin(['Положит.', 'Положительно', '+'])]

        # Создадим таблицу с пациентами, у которых отрицательные анализы
        df_negative_analysis = df_merged[df_merged['Значение'].isin(['Отриц.', 'Отр', '-'])]

        # Уберем из общей таблицы пациентов с положительными и отрицательными анализами
        df_number_meaning = df_merged.drop(df_positive_analysis.index)
        df_number_meaning = df_number_meaning.drop(df_negative_analysis.index)
        # Приведем столбец 'Значение' к типу float
        df_number_meaning['Значение'] = df_number_meaning['Значение'].astype(float)

        # Создадим таблицы пациентов с повышенными и пониженными результатами анализов
        df_enhanced_analysis = df_number_meaning[df_number_meaning['Значение'] > df_number_meaning['max_value']]
        df_reduced_analysis = df_number_meaning[df_number_meaning['Значение'] < df_number_meaning['min_value']]

        # Создадим список, в который занесем результаты анализов из предыдущих таблиц 
        result_analysis = []
        for _, row in df_positive_analysis.iterrows():
            result_analysis.append('Положительный')
        
        for _, row in df_enhanced_analysis.iterrows():
            result_analysis.append('Повышен')
        
        for _, row in df_reduced_analysis.iterrows():
            result_analysis.append('Понижен')

        # Объединим таблицы с повышенными, пониженными и положительными результатами анализов в одну таблицу
        # У нас получилась таблица с пациентами, у которых не в порядке анализы
        df_total_analysis = pd.concat([df_positive_analysis, df_enhanced_analysis])
        df_total_analysis = pd.concat([df_total_analysis, df_reduced_analysis])
        # Добавим к ней столбец с результатми анализов
        df_total_analysis['Заключение'] = result_analysis 
        
        # Создадим промежуточные списки с кодами пациентов, чтобы определить у каких пациентов не в порядке
        # два и более анализа
        # Если в этом списке код пациента оказывается два и более раз, то этот
        # код пациента помещается в список result_list
        # Таким образом мы узнаем у каких пациентов не в порядке два и более анализа
        intermediate_list = list()
        result_list = list()
        for _, row in df_total_analysis.iterrows():
            acc_id = row['Код пациента']
            if acc_id not in intermediate_list: intermediate_list.append(acc_id)
            else:
                result_list.append(acc_id)
        
        # Выберем из всех пациентов, у которых не в порядке анализы только тех,
        # у кого не в порядке два и более анализа
        df_result_patients = df_total_analysis[df_total_analysis['Код пациента'].isin(result_list)]
        
        # Собираем итоговый DataFrame
        output_data = []

        for _, row in df_result_patients.iterrows():
            acc_id = row['Код пациента']
            output_data.append({
                'ФИО пациента': row['name'],
                'Телефон': row['phone'],
                'Название анализа': row['Название анализа'],
                'Заключение': row['Заключение']
            })

        df_output = pd.DataFrame(output_data)

        # Отсортируем фамилии пациентов по алфавиту
        df_output = df_output.sort_values('ФИО пациента')
        # Сохранение результата в Excel
        df_output.to_excel(OUTPUT_EXCEL_FILE, index=False)
        print(f"Результат сохранен в файл: {OUTPUT_EXCEL_FILE}")

    except FileNotFoundError:
        print(f"Ошибка: Файл {INPUT_EXCEL_FILE} не найден.")
    except Error as e:
        print(f"Ошибка при работе с базой данных: {e}")
    except Exception as e:
        print(f"Произошла непредвиденная ошибка: {e}")
    finally:
        # Закрытие соединения
        if conn:
            conn.close()
            print("Соединение с PostgreSQL закрыто")

if  __name__ == "__main__":
    main()



Соединение с базой данных PostgreSQL установлено


C:\Users\Maks\AppData\Local\Temp\ipykernel_9616\3595488634.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_med_an_name = pd.read_sql('SELECT id, name, is_simple, min_value, max_value FROM med_an_name', conn).fillna(0)
C:\Users\Maks\AppData\Local\Temp\ipykernel_9616\3595488634.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_med_name = pd.read_sql('SELECT id, name, phone FROM med_name', conn)


Результат сохранен в файл: result_hard.xlsx
Соединение с PostgreSQL закрыто
